In [58]:
import numpy as np
import datetime
import time
import sqlite3

from binance.spot import Spot

api_endpoint = 'https://api.binance.us'
symbol = 'BNBUSDT'

interval_code = '1m'
interval_ms = 1000*60

end_datetime = datetime.datetime.now()
end_timestamp = time.mktime(end_datetime.timetuple()) * 1000 # In milliseconds 
end_timestamp = int(end_timestamp)

start_datetime = end_datetime - datetime.timedelta(hours=1)
start_timestamp = time.mktime(start_datetime.timetuple()) * 1000 # In milliseconds 
start_timestamp = int(start_timestamp) 

client = Spot(base_url=api_endpoint)

# Get last 10 klines of BNBUSDT at 1h interval
"""
[
  [
    1499040000000,      // Kline open time [0]
    "0.01634790",       // Open price [1]
    "0.80000000",       // High price [2]
    "0.01575800",       // Low price [3]
    "0.01577100",       // Close price [4]
    "148976.11427815",  // Volume [5]
    1499644799999,      // Kline Close time [6]
    "2434.19055334",    // Quote asset volume [7]
    308,                // Number of trades [8]
    "1756.87402397",    // Taker buy base asset volume [9]
    "28.46694368",      // Taker buy quote asset volume [10]
    "0"                 // Unused field, ignore. [11]
  ]
]
"""
k_line_data = client.klines(symbol, interval_code, startTime=start_timestamp, endTime=end_timestamp, limit=100)
k_line_data = np.array(k_line_data)

connection = sqlite3.connect("../database/ticker_data.db")
cursor = connection.cursor()

sql_check_spec = '''SELECT COUNT(*) FROM spot_entry WHERE symbol = ? and ticker_time = ?'''
sql_insert_spec = '''INSERT INTO spot_entry(symbol, ticker_time, open_price, close_price, min_price, max_price, volume)
        VALUES(?, ?, ?, ?, ?, ?, ?)'''

for k_line in k_line_data:   
        ticker_time = k_line[0]
        sql_check_val = (symbol,) + (ticker_time,)

        cursor.execute(sql_check_spec, sql_check_val)
        counts = cursor.fetchall()
        
        if not counts[0][0]:
                sql_insert_val = (symbol,) + tuple(k_line[[0, 1, 4, 3, 2, 5]])
                cursor.execute(sql_insert_spec, sql_insert_val)

connection.commit()
cursor.close()

